# İŞBİRLİKÇİ FİLTRELEME
# # Item-based collaborative Filtering  
# # user-based collaborative Filtering
- x kişinin izlediği filmlere göre x kişisine en benzer davranış gösteren y kişilerinin seçip; 
- y kişiinin izlediği, beğendiği filmleri x kişisnine önermek
# # model-based collaborative Filtering

# PROJE  used-based collaborative filtering
#online bir film izleme platformu var ve kullanıcılara kullanıcıların beğenmeleri üzerinde özel film önerilerinde bulunmak istiyor. 
x kişisi ile aynı film zevkine benzer olan 20 kişi seçilip o kişilerin beğendiği filmler x kişisine önerilmek isteniyor.
odak kullanıcıların biribine benzemesidir. 
* movie.csv
    movield= eşsiz film numarası (UnıquelID) 
    title= film adı
* rating csv
    userid= eşsiz kullanıcı numarası (UnıquelID)
    movield= eşsiz film numaraı (UnıquelID)
    rating= kullanıcı tarafından filme verilen puan
    timestamp= değerlendirme tarihi

1. Veri Setinin Hazırlanması

In [62]:
import pandas as pd

In [63]:
def create_user_movie_df():
    movie= pd.read_csv("/kaggle/input/movielense20m/movie.csv")  # veri seti okutma
    rating= pd.read_csv("/kaggle/input/movielense20m/rating.csv")  #veri seti okutma
    df= movie.merge(rating, how = "left", on= "movieId")  # merge ile iki csv dosyası birleştiriliyor. 
    comment_counts= pd.DataFrame(df["title"].value_counts()) # oy sayılarının sayıldığı dataframe 
    rare_movies= comment_counts[comment_counts["title"]<= 1000].index   # az kere puanlanmış sahip olan filmler 
    common_movies= df[~df["title"].isin(rare_movies)]   # puanlası az olan filmlerin veriden temizlenmei 
    user_movie_df= common_movies.pivot_table(index= "userId", columns= "title", values="rating")  # veri yapısı fimller arası korelasyon uygulayabileceğiiz hale getirilmesi 
    return user_movie_df

    

In [64]:
user_movie_df = create_user_movie_df()

In [65]:
user_movie_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
random_user= int(pd.Series(user_movie_df.index).sample(1, random_state=45).values) 
#kullanıcılardan random kişi seçerek çalışacağız
print(random_user)

28941


2. İzlenen Filmleri Getirme Uygulaması 

In [67]:
random_user_df= user_movie_df[user_movie_df.index == random_user]
print(random_user_df) 
#  random user "28941" verilerini aldık

title    'burbs, The (1989)  (500) Days of Summer (2009)  \
userId                                                     
28941.0                 NaN                          NaN   

title    *batteries not included (1987)  ...And Justice for All (1979)  \
userId                                                                   
28941.0                             NaN                            NaN   

title    10 Things I Hate About You (1999)  10,000 BC (2008)  \
userId                                                         
28941.0                                NaN               NaN   

title    101 Dalmatians (1996)  \
userId                           
28941.0                    NaN   

title    101 Dalmatians (One Hundred and One Dalmatians) (1961)  \
userId                                                            
28941.0                                                NaN        

title    102 Dalmatians (2000)  12 Angry Men (1957)  ...  \
userId                                

In [68]:
movies_watched= random_user_df.columns[random_user_df.notna().any()].tolist()
# #  random user "28941" verilerinden ilgili kullanıcının izlemediği filmler çıkarıldı notna().any() ile 
# ve çıktıyı liste yaptık
print(movies_wached)

['Ace Ventura: Pet Detective (1994)', 'Ace Ventura: When Nature Calls (1995)', 'Aladdin (1992)', 'American President, The (1995)', 'Apollo 13 (1995)', 'Babe (1995)', 'Bullets Over Broadway (1994)', 'Clueless (1995)', 'Disclosure (1994)', 'Forrest Gump (1994)', 'Four Weddings and a Funeral (1994)', 'Home Alone (1990)', 'Jurassic Park (1993)', 'Like Water for Chocolate (Como agua para chocolate) (1992)', 'Little Women (1994)', "Mr. Holland's Opus (1995)", 'Mrs. Doubtfire (1993)', 'Much Ado About Nothing (1993)', "Muriel's Wedding (1994)", 'Nine Months (1995)', 'Operation Dumbo Drop (1995)', 'Piano, The (1993)', 'Postman, The (Postino, Il) (1994)', 'Ready to Wear (Pret-A-Porter) (1994)', 'Remains of the Day, The (1993)', 'Sabrina (1995)', "Schindler's List (1993)", 'Secret Garden, The (1993)', 'Sense and Sensibility (1995)', 'Shadowlands (1993)', 'Silence of the Lambs, The (1991)', 'Star Trek: Generations (1994)', 'Stargate (1994)']


In [69]:
len(movies_wached)
# kullancıının izlediği film sayısı 

33

In [70]:
user_movie_df.loc[user_movie_df.index== random_user,
                 user_movie_df.columns== 'Ace Ventura: Pet Detective (1994)']
# kullanıcının filme veridği puanı kontrol etme

title,Ace Ventura: Pet Detective (1994)
userId,
28941.0,3.0


3. Aynı Filmleri İzleyen Diğer Kullancııların Veri ve ID'lerine Erişmek

In [71]:
movies_watched_df= user_movie_df[movies_wached]  # random userın izlediği filmlerin genel puanlaması 
print(movies_wached_df)

title     Ace Ventura: Pet Detective (1994)  \
userId                                        
1.0                                     NaN   
2.0                                     NaN   
3.0                                     NaN   
4.0                                     NaN   
5.0                                     NaN   
...                                     ...   
138489.0                                NaN   
138490.0                                NaN   
138491.0                                NaN   
138492.0                                NaN   
138493.0                                4.0   

title     Ace Ventura: When Nature Calls (1995)  Aladdin (1992)  \
userId                                                            
1.0                                         NaN             NaN   
2.0                                         NaN             NaN   
3.0                                         NaN             NaN   
4.0                                         3.0      

In [72]:
# Kullanıcı ile en az 20 film izleyen kullancıları dikkate alacağız. 
# her kullanıcının kaç film izlediğine bakacağız 
user_movie_count= movies_wached_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
print(user_movie_count)

          userId   0
0            1.0   1
1            2.0   2
2            3.0   4
3            4.0   6
4            5.0  11
...          ...  ..
138488  138489.0   1
138489  138490.0   7
138490  138491.0   0
138491  138492.0   2
138492  138493.0   9

[138493 rows x 2 columns]


In [73]:
user_movie_count.columns = ["userId", "movie_count"]

In [74]:
user_movie_count[user_movie_count["movie_count"]> 20].sort_values("movie_count", ascending= False)

,userId,movie_count
94230,94231.0,33
100398,100399.0,33
118204,118205.0,33
15918,15919.0,33
124051,124052.0,33
...,...,...
79214,79215.0,21
79174,79175.0,21
9105,9106.0,21
78515,78516.0,21


In [75]:
user_movie_count[user_movie_count["movie_count"] == 33].count()
# kullanıcı ile 33 filmin hepsini izleyen kullanıcılr

userId         17
movie_count    17
dtype: int64

In [76]:
users_same_movies= user_movie_count[user_movie_count["movie_count"]> 20]["userId"]
print(users_same_movies)

129          130.0
155          156.0
157          158.0
183          184.0
294          295.0
            ...   
138207    138208.0
138278    138279.0
138381    138382.0
138414    138415.0
138482    138483.0
Name: userId, Length: 3202, dtype: float64


In [77]:
# adet yerine oran olarak da gidilebilir. 
# perch = len(movies_wached)* 60/100
#sers_same_movies= user_movie_count[user_movie_count["movie_count"]> perch]["userID"]

4. Öneri Yapılacak Kullanıcı ile En Benzrr Davranışlı Kullanıcıların Belirlenmesi 
    1. 28941 kişi ve diğer kullancııların vrilerini bir araya getirme
    2. Korelasyon df'ni oluşturma
    3. En benzer (Top users) kullanıcıları bulma

In [78]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])  # aynı filmi izleyenler ve 3 filmi birletiriyoruz 
print(final_df)
#28941 kulancıı ile diğer kulllanıcıların verilerini bir araya getirdik. 

title     Ace Ventura: Pet Detective (1994)  \
userId                                        
130.0                                   4.0   
156.0                                   3.0   
158.0                                   2.0   
184.0                                   2.0   
295.0                                   NaN   
...                                     ...   
138279.0                                3.0   
138382.0                                1.0   
138415.0                                1.0   
138483.0                                1.0   
28941.0                                 3.0   

title     Ace Ventura: When Nature Calls (1995)  Aladdin (1992)  \
userId                                                            
130.0                                       3.0             NaN   
156.0                                       NaN             NaN   
158.0                                       1.0             4.0   
184.0                                       3.0      

In [79]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
# final_df'nin trasnpozunu aldık(T), corelasyonunu aldık(corr), pivotunu aldık(unsatck), birden fazla tektatlanan silindi(drop_duplicates)
print(corr_df)

userId    userId 
28866.0   67756.0   -0.936065
60562.0   37121.0   -0.915003
34103.0   80593.0   -0.898718
62575.0   21398.0   -0.896612
117826.0  48416.0   -0.883969
                       ...   
96952.0   34350.0    0.971667
32344.0   68063.0    0.977387
28941.0   28941.0    1.000000
16562.0   41005.0    1.000000
725.0     43765.0         NaN
Length: 3821114, dtype: float64


In [80]:
corr_df= pd.DataFrame(corr_df, columns= ["corr"])
corr_df.index.names = ["user_id_1", "user_id_2"]
corr_df = corr_df.reset_index()
print(corr_df)  # tüm kullanılar ile olan korelasyolar

         user_id_1  user_id_2      corr
0          28866.0    67756.0 -0.936065
1          60562.0    37121.0 -0.915003
2          34103.0    80593.0 -0.898718
3          62575.0    21398.0 -0.896612
4         117826.0    48416.0 -0.883969
...            ...        ...       ...
3821109    96952.0    34350.0  0.971667
3821110    32344.0    68063.0  0.977387
3821111    28941.0    28941.0  1.000000
3821112    16562.0    41005.0  1.000000
3821113      725.0    43765.0       NaN

[3821114 rows x 3 columns]


In [83]:
# bizim ihtiyacımız 28941 ıd'li kullanıcyla korelasyona bakmak. 
top_users= corr_df[(corr_df["user_id_1"]== random_user) & (corr_df["corr"]>= 0.65)][ # user_id_1 i 28941 ıd li neri vereceğimiz kişiyi yaptık, corelasyonu 0.65'ten büyük olanları seçtik. 
    ["user_id_2", "corr"]].reset_index(drop=True)  #"user_id_2", "corr" sütunlarını istiyorum dedik, ve indexi resetledik. 
print(top_users)


    user_id_2      corr
0    132307.0  0.660726
1     42696.0  0.661268
2      1362.0  0.665461
3    124546.0  0.669585
4     73844.0  0.670105
5     66603.0  0.673862
6     58705.0  0.680166
7    131285.0  0.681491
8    125901.0  0.681795
9     91954.0  0.696581
10    91954.0  0.696581
11    66175.0  0.698099
12    66175.0  0.698099
13   121484.0  0.699161
14    24196.0  0.699198
15    10943.0  0.703123
16    60411.0  0.727636
17     2171.0  0.728516
18    67346.0  0.732943
19    67346.0  0.732943
20   136259.0  0.737177
21   136259.0  0.737177
22    93089.0  0.738536
23     9783.0  0.747942
24   127259.0  0.763925
25   101628.0  0.790405
26    45158.0  0.800749
27    28941.0  1.000000


In [86]:
top_users= top_users.sort_values(by="corr", ascending=False)
print(top_users)
# birinci sıradaki hedef kullancının kendisi 

    user_id_2      corr
27    28941.0  1.000000
26    45158.0  0.800749
25   101628.0  0.790405
24   127259.0  0.763925
23     9783.0  0.747942
22    93089.0  0.738536
21   136259.0  0.737177
20   136259.0  0.737177
19    67346.0  0.732943
18    67346.0  0.732943
17     2171.0  0.728516
16    60411.0  0.727636
15    10943.0  0.703123
14    24196.0  0.699198
13   121484.0  0.699161
12    66175.0  0.698099
11    66175.0  0.698099
10    91954.0  0.696581
9     91954.0  0.696581
8    125901.0  0.681795
7    131285.0  0.681491
6     58705.0  0.680166
5     66603.0  0.673862
4     73844.0  0.670105
3    124546.0  0.669585
2      1362.0  0.665461
1     42696.0  0.661268
0    132307.0  0.660726


In [87]:
top_users.rename(columns = {"user_id_2": "userId"}, inplace=True)
print(top_users)

      userId      corr
27   28941.0  1.000000
26   45158.0  0.800749
25  101628.0  0.790405
24  127259.0  0.763925
23    9783.0  0.747942
22   93089.0  0.738536
21  136259.0  0.737177
20  136259.0  0.737177
19   67346.0  0.732943
18   67346.0  0.732943
17    2171.0  0.728516
16   60411.0  0.727636
15   10943.0  0.703123
14   24196.0  0.699198
13  121484.0  0.699161
12   66175.0  0.698099
11   66175.0  0.698099
10   91954.0  0.696581
9    91954.0  0.696581
8   125901.0  0.681795
7   131285.0  0.681491
6    58705.0  0.680166
5    66603.0  0.673862
4    73844.0  0.670105
3   124546.0  0.669585
2     1362.0  0.665461
1    42696.0  0.661268
0   132307.0  0.660726


In [91]:
rating= pd.read_csv("/kaggle/input/movielense20m/rating.csv")  # rating veri setini aldık
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how="inner")  # userId'ye göre birleştirme yaptık (movieId ve rating eklettik. )
print(top_users_rating)

         userId      corr  movieId  rating
0       28941.0  1.000000        7     5.0
1       28941.0  1.000000       11     3.0
2       28941.0  1.000000       17     5.0
3       28941.0  1.000000       19     2.0
4       28941.0  1.000000       34     5.0
...         ...       ...      ...     ...
13680  132307.0  0.660726      593     4.0
13681  132307.0  0.660726      595     4.0
13682  132307.0  0.660726      597     4.0
13683  132307.0  0.660726      608     5.0
13684  132307.0  0.660726     1084     3.0

[13685 rows x 4 columns]


In [94]:
top_users_ratings= top_users_ratings[top_users_ratings["userId" ] != random_user] 
# ilk satırdaki öneri yapacağımız kullanıcının kendiini çıkardık. 
print(top_users_ratings)

         userId      corr  movieId  rating
33      45158.0  0.800749        1     1.5
34      45158.0  0.800749        3     1.0
35      45158.0  0.800749       17     3.5
36      45158.0  0.800749       19     2.0
37      45158.0  0.800749       22     3.0
...         ...       ...      ...     ...
13680  132307.0  0.660726      593     4.0
13681  132307.0  0.660726      595     4.0
13682  132307.0  0.660726      597     4.0
13683  132307.0  0.660726      608     5.0
13684  132307.0  0.660726     1084     3.0

[13652 rows x 4 columns]


5. weighted Average Recommendation Scoru'un Hesaplanması 

In [96]:
top_users_ratings["weihted_rating"]= top_users_ratings["corr"]* top_users_ratings["rating"] # rating ve corelasyonu çarpıp rating ağırlığını bulduk
recommendation_df= top_users_ratings.groupby("movieId").agg({"weihted_rating": "mean"})  #movieıd'ye göre groupb yaparak ağırlıklandırılmış ortalama ratingleri bulduk
recommendation_df =recommendation_df.reset_index() 3 indexi resetledik 
print(recommendation_df)

      movieId  weihted_rating
0           1        2.205824
1           2        1.463950
2           3        1.181772
3           4        1.642769
4           5        1.151408
...       ...             ...
4069    56367        2.565300
4070    56508        2.931772
4071    56788        2.565300
4072    58303        2.931772
4073    59725        2.931772

[4074 rows x 2 columns]


In [99]:
recommendation_df[recommendation_df["weihted_rating"] > 3.5 ]
movies_to_be_recommed= recommendation_df[recommendation_df["weihted_rating"]> 3.5].sort_values("weihted_rating" ,ascending=False)
print(movies_to_be_recommed)

      movieId  weihted_rating
26         30        3.952023
257       326        3.952023
183       242        3.952023
45         53        3.952023
920      1348        3.692678
394       501        3.679739
3873    25850        3.664714
3691     8128        3.664714
3907    26394        3.664714
3622     7585        3.664714
3610     7490        3.664714
3548     7215        3.664714
3502     7087        3.664714
3410     6837        3.664714
3195     6235        3.664714
3183     6216        3.664714
3064     5799        3.664714
2752     4920        3.664714
2721     4840        3.664714
2563     4495        3.664714
2396     4117        3.664714
3966    32234        3.664714
3121     5992        3.657336
201       263        3.649974
2482     4334        3.642581
3420     6870        3.642581
3439     6954        3.642581
3445     6974        3.642581
1578     2570        3.642581
1543     2496        3.642581
799      1201        3.642581
848      1257        3.603372
2636     4

In [100]:
movie= pd.read_csv("/kaggle/input/movielense20m/movie.csv")  # veri seti okutma
movies_to_be_recommed = movies_to_be_recommed.merge(movie[["movieId", "title"]])
print(movies_to_be_recommed)

    movieId  weihted_rating                                              title
0        30        3.952023  Shanghai Triad (Yao a yao yao dao waipo qiao) ...
1       326        3.952023                            To Live (Huozhe) (1994)
2       242        3.952023                      Farinelli: il castrato (1994)
3        53        3.952023                                    Lamerica (1994)
4      1348        3.692678  Nosferatu (Nosferatu, eine Symphonie des Graue...
5       501        3.679739                                       Naked (1993)
6     25850        3.664714                                     Holiday (1938)
7      8128        3.664714                       Au revoir les enfants (1987)
8     26394        3.664714                          Turning Point, The (1977)
9      7585        3.664714                                  Summertime (1955)
10     7490        3.664714  At First Sight (Entre Nous) (Coup de foudre) (...
11     7215        3.664714                        T

Çalışmanın Scriptinin Oluştrulması

In [ ]:
def create_user_movie_df():
    movie= pd.read_csv("/kaggle/input/movielense20m/movie.csv")  # veri seti okutma
    rating= pd.read_csv("/kaggle/input/movielense20m/rating.csv")  #veri seti okutma
    df= movie.merge(rating, how = "left", on= "movieId")  # merge ile iki csv dosyası birleştiriliyor. 
    comment_counts= pd.DataFrame(df["title"].value_counts()) # oy sayılarının sayıldığı dataframe 
    rare_movies= comment_counts[comment_counts["title"]<= 1000].index   # az kere puanlanmış sahip olan filmler 
    common_movies= df[~df["title"].isin(rare_movies)]   # puanlası az olan filmlerin veriden temizlenmei 
    user_movie_df= common_movies.pivot_table(index= "userId", columns= "title", values="rating")  # veri yapısı fimller arası korelasyon uygulayabileceğiiz hale getirilmesi 
    return user_movie_df

In [108]:
def user_based_recommendation(random_user, user_movie_df, ratio=60, cor_th=0.60, score=3.5):
    import pandas as pd
    random_user_df= user_movie_df[user_movie_df.index == random_user] # random userın tüm verileini aldık
    #random user "28941" verilerinden ilgili kullanıcının izlemediği filmler çıkarıldı notna().any() ile ve listeye çevrildi. 
    movies_watched= random_user_df.columns[random_user_df.notna().any()].tolist()
    # yukarıda seçtiğimiz random userın izlediği filmlerin diğer kullancılarla olan verisi ve ratingi eklendi user_movie_df'den alındı. 
    movies_watched_df= user_movie_df[movies_wached]
     # random kullancıının izlediği filmler arasında diğer kullanıcıların kaç fil ilzediği hesapldık
    user_movie_count= movies_wached_df.T.notnull().sum()
    user_movie_count = user_movie_count.reset_index()  # resette çıkardık 
    user_movie_count.columns = ["userId", "movie_count"] # kolonların isimlerini günceedik. 
    #random kulanıcı ile benzer film ilzyenlerin %60 ını aldık, defaut tanımladık fonskiyonu tanımlarken
    perc= len(movies_watched)*ratio/100 #orandan çıkacak film adet
    users_same_mivies= user_movie_count[user_movie_count["movie_count"]> perc].sort_values("movie_count", ascending= False) # adede göre filtreleme
    
    #random kullanıcı benzer filmlerin olduğu users_same_movie ile random_userın izlediği filmlrin listelerini birleştirdik. 
    final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]]) 
    
    #final_df'nin trasnpozunu aldık(T), corelasyonunu aldık(corr), pivotunu aldık(unsatck), birden fazla tekrarlanan silindi(drop_duplicates)
    corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
    # tüm kullanılar ile olan korelasyolar
    corr_df= pd.DataFrame(corr_df, columns= ["corr"]) # dataFrame çevirdik
    corr_df.index.names = ["user_id_1", "user_id_2"] # kolon isimşerini günceleldik
    corr_df = corr_df.reset_index() # indexi resetledik
    
    #random user ile en benzer kullanıcıları bulma
    # user_id_1 i random_user vereceğimiz kişiyi yaptık, corelasyonu 0.60'ten büyük olanları seçtik. user_id_2", "corr" sütunlarını istiyorum dedik, ve indexi resetledik.
    top_users= corr_df[(corr_df["user_id_1"]== random_user) & (corr_df["corr"]>=cor_th )][["user_id_2", "corr"]].reset_index(drop=True) 
    top_users= top_users.sort_values(by="corr", ascending=False)  # top usersları random user ile korelasyonuna göre sıraladık 
    top_users.rename(columns = {"user_id_2": "userId"}, inplace=True)  # kolon isimlerini güncelledik
    
     #rating veri setini aldık, userId'ye göre birleştirme yaptık (movieId ve rating eklettik.
    rating= pd.read_csv("/kaggle/input/movielense20m/rating.csv")  
    top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how="inner") 
    top_users_ratings= top_users_ratings[top_users_ratings["userId" ] != random_user]  #veriden random_userı çıkardık
    
    # top_usersların ratingi iel korelasyonu çarpıp apırlıklandırılmış ranting bulundu, rantinglerin group by ile ortalaması alındı
    top_users_ratings["weihted_rating"]= top_users_ratings["corr"]* top_users_ratings["rating"]
    recommendation_df= top_users_ratings.groupby("movieId").agg({"weihted_rating": "mean"})
    recommendation_df =recommendation_df.reset_index()  #groupby'dan sonra reset_index yapıdlı 
    
    # tavsiye edilen filmlerden ratingi fonskionun başnda default verilen score değerinden büyük olanları aldık--> movies_to_be_recommed
    #burada tavsiye edilen filmlerin listesi oluşmuş oldu. 
    movies_to_be_recommed= recommendation_df[recommendation_df["weihted_rating"]> score].sort_values("weihted_rating" ,ascending=False)
    
    #tavsiye edilen filmlein ıd'si ile çalışmıştık movie dosyasını okutup, ıd'ye karılık gelen filmleri getirdik. 
    movie= pd.read_csv("/kaggle/input/movielense20m/movie.csv")  # veri seti okutma
    movies_to_be_recommed = movies_to_be_recommed.merge(movie[["movieId", "title"]])

    return movies_to_be_recommed
    



In [105]:
random_user= int(pd.Series(user_movie_df.index).sample(1).values) 
print(random_user)

95518


In [109]:
user_based_recommendation(random_user, user_movie_df)

,movieId,weihted_rating,title
0,109953,4.930584,"Cat Concerto, The (1947)"
1,47721,4.930584,"Red Balloon, The (Ballon rouge, Le) (1956)"
2,632,4.719399,Land and Freedom (Tierra y libertad) (1995)
3,62383,4.702948,"20,000 Leagues Under the Sea (1916)"
4,3851,4.666283,I'm the One That I Want (2000)
...,...,...,...
266,3307,3.505278,City Lights (1931)
267,1260,3.503768,M (1931)
268,1184,3.502877,Mediterraneo (1991)
269,8459,3.502440,"Heiress, The (1949)"


In [112]:
# 95518 ıd'li kullanıcıya çok fazla film önerisinde bulunduk bunu azaltamk için score ve threshold değerlerinde oynama yaoabiliriz, ilk userımız daha az film izliyormuç 
user_based_recommendation(95518, user_movie_df, ratio=0.70, cor_th=0.70, score= 4)
# tavsiye edile film sayısı 55'e düştü 

,movieId,weihted_rating,title
0,109953,4.930584,"Cat Concerto, The (1947)"
1,8270,4.930584,"Hairdresser's Husband, The (Le mari de la coif..."
2,47721,4.930584,"Red Balloon, The (Ballon rouge, Le) (1956)"
3,632,4.719399,Land and Freedom (Tierra y libertad) (1995)
4,62383,4.702948,"20,000 Leagues Under the Sea (1916)"
5,3851,4.666283,I'm the One That I Want (2000)
6,1216,4.566479,"Big Blue, The (Grand bleu, Le) (1988)"
7,4194,4.496144,I Know Where I'm Going! (1945)
8,95311,4.437526,Presto (2008)
9,106920,4.437526,Her (2013)


In [110]:
random_user= int(pd.Series(user_movie_df.index).sample(1).values)  # herhangi random_user seçtiriyorum 
user_based_recommendation(random_user, user_movie_df)

,movieId,weihted_rating,title
0,33779,4.524811,Eddie Izzard: Dress to Kill (1999)
1,32657,4.524811,"Man Who Planted Trees, The (Homme qui plantait..."
2,5615,4.475351,Invincible (2001)
3,8848,4.266601,"Vitelloni, I (a.k.a. The Young and the Passion..."
4,25971,4.200134,Carrie (1952)
...,...,...,...
104,5262,3.507073,Mr. Sardonicus (1961)
105,34643,3.507073,"Three Stooges Meet Hercules, The (1962)"
106,80141,3.501150,"Christmas Toy, The (1986)"
107,72386,3.501150,Broken Embraces (Los abrazos rotos) (2009)
